In [ ]:
import os
import json
from concurrent.futures import ProcessPoolExecutor
from text_image_generator import create_text_image

In [ ]:
os.makedirs("./tmp", exist_ok=True)


def get_all_fonts(parent_folder="fonts", exclude_bold=True, exclude_italic=True):
    import os

    font_files = []
    for root, dirs, files in os.walk(parent_folder):
        for file in files:
            if file.endswith(".ttf") or file.endswith(".otf"):
                font_files.append(os.path.join(root, file))

    if exclude_bold:
        font_files = [f for f in font_files if "bold" not in f.lower()]

    if exclude_italic:
        font_files = [f for f in font_files if "italic" not in f.lower()]

    print(f"Found {len(font_files)} fonts")

    return font_files


def generate_dataset(word, fonts, output_folder, scale_factor=0.6):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    configurations = []

    tasks = []

    with ProcessPoolExecutor() as executor:
        for font_path in fonts:
            # for circle_index in range(0, len(word)):
            for circle_index in [0]:
                for thickness in [4]:
                # for thickness in range(4, 7, 1):
                    for x_offset in range(0, 1, 1):
                        for y_offset in range(0, 1, 1):
                            for padding in [
                                25
                            ]:
                                config = {
                                    "word": word,
                                    "font_path": font_path,
                                    "circle_index": circle_index,
                                    "thickness": thickness,
                                    "scale_factor": scale_factor,
                                    "padding": padding,
                                    "x_offset": x_offset,
                                    "y_offset": y_offset,
                                    "canvas_width": 10,
                                    "canvas_height": 2,
                                    "final_width": 512,
                                    "final_height": 512,
                                }
                                tasks.append(
                                    (
                                        config,
                                        executor.submit(
                                            create_text_image,
                                            text=word,
                                            font_path=font_path,
                                            circle_index=circle_index,
                                            thickness=thickness,
                                            scale_factor=scale_factor,
                                            padding=padding,  # Use variable padding
                                            x_offset=x_offset,
                                            y_offset=y_offset,
                                            canvas_width=10,
                                            canvas_height=2,
                                            final_width=512,
                                            final_height=512,
                                            output_folder=output_folder,
                                        ),
                                    )
                                )

        for config, task in tasks:
            image_path = task.result()
            if image_path:
                config["image_path"] = image_path
                configurations.append(config)

    # Save the configurations to a JSON file
    with open(os.path.join(output_folder, "configurations.json"), "w") as f:
        json.dump(configurations, f, indent=4)

In [ ]:
fonts = get_all_fonts(parent_folder="fonts")
# words = ["Acknowledgement", "Subdermatoglyphic", "tHyUiKaRbNqWeOpXcZvM"]
words = ["Acknowledgement", "Subdermatoglyphic", "tHyUiKaRbNqWeOpXcZvM"]

for word in words:
    output_folder = f"./images/{word}"
    generate_dataset(word, fonts, output_folder, scale_factor=1.4)